# Computational Social Choice
Computational Social Choice deals with problems arising from the aggregation of preferences of a group of agents. It has important implications when considering voting systems and elections.
While aggregating personal preferences or computing the winner of an election might seem easy or intuitive, voting systems and aggregation rules are vulnerable to manipulation strategies. In addition, voting aggregation rules should satisfy some desired properties, such as anonimity or being difficult to manipulate.

Computational social choice uses an *axiomatic method* where desired properties are treated as axioms and and consequences are derived from axioms as theorems.
Typically, there are two types of theorems that are derived in computational social choice:
- **Representation theorems** where, given a set of axioms, the goal is to show that a particular class of mechanisms is the only one which satisfies the set of axioms;
- **Impossibility theorems** where, given a set of axioms, the goal is to show that there exists no aggregation rule which satisfies the set of axioms.

In this lab, we are going to study some voting aggregation rules, ranging from the most commonly employed to the ones which make the possibility of ties as unlikely as possible and which exhibit the highest resistance to manipulation strategies.

In [ ]:
import random

In [ ]:
random.seed(42)

# Preferences are given as ordered lists of candidates
C = ['A', 'B', 'C', 'D']

# Generate N random permutations of C and we get preferences for N people!
N = 10
preferences = list()
for i in range(N):
    random.shuffle(C)
    preferences.append(C.copy())

for preference in preferences:
    print(preference)

['C', 'B', 'D', 'A']
['A', 'D', 'C', 'B']
['D', 'B', 'C', 'A']
['B', 'C', 'D', 'A']
['C', 'D', 'A', 'B']
['D', 'B', 'A', 'C']
['A', 'B', 'D', 'C']
['B', 'C', 'A', 'D']
['B', 'A', 'C', 'D']
['C', 'D', 'B', 'A']


# Plurality Voting

In [ ]:
# Plurality vote elects the candidate ranked first more often
def plurality_vote(preferences):
    counts = {candidate: 0 for candidate in C}
    for preference in preferences:
      counts[preference[0]] += 1

    winner = max(counts, key = counts.get)
    return winner, counts

In [ ]:
winner, counts = plurality_vote(preferences)
print("The winner is:", winner)

# In this example, we actually have a tie between C and B

print(counts)

The winner is: C
{'C': 3, 'D': 2, 'B': 3, 'A': 2}


# Borda counting

In [ ]:
# Borda counting
def borda_vote(preferences):
    counts = {candidate: 0 for candidate in C}
    borda_votes = [len(preferences[0])-i-1 for i in range(len(preferences[0]))]

    for preference in preferences:
      for i in range(len(preference)):
        counts[preference[i]] += borda_votes[i]

    winner = max(counts, key = counts.get)
    return winner, counts

In [ ]:
winner, counts = borda_vote(preferences)
print("The winner is:", winner)

# Borda method breaks does not have a tie between C and B and declares B the winner
print(counts)

The winner is: B
{'C': 16, 'D': 15, 'B': 18, 'A': 11}


# Instant-runoff Voting

In [ ]:
def has_majority(preferences, counts):
    return max(counts.values()) > len(preferences)//2


# Instant-runoff
def IRV(preferences):
    global C
    C_copy = C.copy()

    # We make a copy of the preferences to avoid messing with the original matrix
    preferences_copy = [preference.copy() for preference in preferences]

    # We reuse the plurality_vote function made before to get the counts
    _, counts = plurality_vote(preferences_copy)

    while not has_majority(preferences_copy, counts):
      candidate_to_remove = min(counts, key = counts.get)
      C.remove(candidate_to_remove)

      # We remove the candidate also from each preference in the preferences list
      preferences_copy = [[candidate for candidate in preference if candidate in C] for preference in preferences_copy]
      _, counts = plurality_vote(preferences_copy)

    # Restore original candidates
    C = C_copy.copy()

    winner = max(counts, key = counts.get)
    return winner, counts

In [ ]:
winner, counts = IRV(preferences)
print("The winner is:", winner)

# Instant-runoff breaks the tie between C and B and declares B the winner
print(counts)

The winner is: B
{'C': 4, 'B': 6}


# Schulze Method

In [ ]:
import networkx as nx
from itertools import permutations
import sys

# Utils
sorted_candidates = C.copy()
sorted_candidates.sort()

## Works only under the assumption that the candidates are always given in the form ['A','B','C',...]
## (i.e., one char to represent the candidate, with subsequent candidates separated by 1 in the ASCII table)
candidate_indexes = {candidate: ord(candidate)-ord(sorted_candidates[0]) for candidate in sorted_candidates}

perms = [perm for perm in permutations(sorted_candidates, 2)]

def get_graph_from_matrix(matrix):
    G = nx.DiGraph()

    for i in range(len(matrix)):
      to_add = []
      for j in range(len(matrix)):
        if i != j:
          to_add.append((sorted_candidates[i],sorted_candidates[j],matrix[i][j]))
      G.add_weighted_edges_from(to_add)

    return G

# We could have made this function compute directly the DiGraph
# But -as the exercise asks- we are gonna make it compute the matrix
def compute_pairwise_matrix(preferences: list):
    matrix = [[0 for _ in range(len(C))] for _ in range(len(C))]

    for preference in preferences:
      for c1, c2 in perms:
        idx_c1 = candidate_indexes[c1]
        idx_c2 = candidate_indexes[c2]
        if preference.index(c1) < preference.index(c2):
          matrix[idx_c1][idx_c2] += 1
        else:   # Can't be equal then the candidate at idx i precedes the one at j
          matrix[idx_c2][idx_c1] += 1
    return matrix


def all_pairs_widest_path(weighted_graph: nx.DiGraph):
    matrix = [[0 for _ in range(len(C))] for _ in range(len(C))]
    for c1, c2 in perms:
      idx_c1 = candidate_indexes[c1]
      idx_c2 = candidate_indexes[c2]

      # If there exists a direct edge, assign its weight to var, otherwise assign 0
      direct_weight = weighted_graph.get_edge_data(c1, c2, default=0)["weight"]

      for path in nx.all_simple_edge_paths(weighted_graph, c1, c2):
        if len(path) == len(sorted_candidates) - 1:   # then it is the widest path
          widest_path_min_weight = int(sys.maxsize)
          for c1, c2 in path:   # every element of a path is an edge (src, dest)
            weight = weighted_graph.get_edge_data(c1, c2)["weight"]
            widest_path_min_weight = min(widest_path_min_weight, weight)
          break   # You can go to the next tuple at this point

      matrix[idx_c1][idx_c2] = max(direct_weight, widest_path_min_weight)
    return matrix


def schulze_method(preferences: list):
    pairwise_matrix = compute_pairwise_matrix(preferences)

    G = get_graph_from_matrix(pairwise_matrix)

    all_pairs_widest_path_matrix = all_pairs_widest_path(G)

    counts = {candidate: sum(all_pairs_widest_path_matrix[candidate_indexes[candidate]]) for candidate in sorted_candidates}
    winner = max(counts, key = counts.get)
    return winner, counts

In [ ]:
winner, counts = schulze_method(preferences)
print("The winner is:", winner)

# The winner is B
print(counts)

The winner is: B
{'A': 24, 'B': 36, 'C': 32, 'D': 30}
